# Install libraries

In [8]:
!pip install beautifulsoup4
!pip install selenium
!pip install pandas
!pip install tqdm
!pip install ipywidgets
!jupyter nbextension enable --py widgetsnbextension
!jupyter labextension install @jupyter-widgets/jupyterlab-manager

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK
Building jupyterlab assets (build:prod:minimize)


# Importer libraries

In [1]:
import time
import shutil
import random
import pandas as pd
from tqdm.notebook import tqdm
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.common.by import By

# Helper functions

In [ ]:
def get_option_values(search_area, name):
    section = search_area.find('select', {'name': name})
    return [option.get_text() for option in section.find_all('option')][1:] # First option is a placeholder so we should remove it

def click_option(search_area, name, value):
    search_area.find_element(By.XPATH, f"//select[@name='{name}']/option[text()='{value}']").click()
    
def download_csv(exporter, exporter_area, importer, importer_area, industry, production, trainding_start_year, trading_end_year):
    # Select all options
    search_area = driver.find_element_by_id('search-area')
    click_option(search_area, 'exp', exporter)
    click_option(search_area, 'ex_area', exporter_area)
    click_option(search_area, 'imp', importer)
    click_option(search_area, 'im_area', importer_area)
    click_option(search_area, 'industry', industry)
    click_option(search_area, 'proc', production)
    click_option(search_area, 'syear', trainding_start_year)
    click_option(search_area, 'eyear', trading_end_year)
    
    # Click Search button
    search_area.find_element(By.XPATH, "//button").click()

    time.sleep(10)
    
    # Click output CSV file button
    driver.find_element(By.XPATH, "//a[@href='tcsv.php']").click()
    
    time.sleep(10)
    # Move file for checking uniqueness
    filepath = f'data/{exporter}_{exporter_area}_{importer}_{importer_area}_{industry}_{production}.csv'.replace(' ', '_')
    shutil.move('data/tid.csv', filepath)

# Create selenium driver

In [3]:
chrome_options = webdriver.ChromeOptions()
prefs = {"download.default_directory": "/data"}
chrome_options.add_experimental_option("prefs", prefs)

driver = webdriver.Remote(
    command_executor='http://selenium-hub:4444/wd/hub',
    desired_capabilities=DesiredCapabilities.CHROME,
    options=chrome_options)
driver.get("https://www.rieti-tid.com/trade.php")
soup = bs(driver.page_source, 'html.parser')

# Get all options

In [4]:
search_area = soup.find('section', {'id': 'search-area'})

exporters = get_option_values(search_area, 'exp')
exporter_areas = get_option_values(search_area, 'ex_area')
importers = get_option_values(search_area, 'imp')
importer_areas = get_option_values(search_area, 'im_area')
industries = get_option_values(search_area, 'industry')
productions = get_option_values(search_area, 'proc')
# stage = get_option_values(search_area, 'stage')
trading_start_years = get_option_values(search_area, 'syear')
trading_end_years = get_option_values(search_area, 'eyear')

# Download all csv files

In [25]:
trading_start_year = trading_start_years[0]
trading_end_year = trading_end_years[-1]
for exporter in tqdm(exporters, desc='exporter'):
    for importer in tqdm(importers, desc='importers'):
        if exporter == importer:
            continue
        for exporter_area in tqdm(exporter_areas, desc='exporter_areas'):
            for importer_area in tqdm(importer_areas, desc='importer_areas'):
                for industry in tqdm(industries, desc='industries'):
                    for production in tqdm(productions, desc='productions'):
                        download_csv(exporter, exporter_area, importer, importer_area, industry, production, trading_start_year, trading_end_year) 
                        # Wait for random seconds (Up to 5 sec)
                        wait_sec = random.random() * 100 % 5
                        time.sleep(wait_sec)

KeyboardInterrupt: 

In [26]:
driver.close()

# Concatenate csv files

In [27]:
import pandas as pd
import glob

In [28]:
paths = glob.glob('data/*.csv')

In [36]:
dfs = []
for path in paths:
    df = pd.read_csv(path)
    dfs.append(df)

concat_df = pd.concat(dfs)

In [37]:
concat_df

,Year,Industry,Production Stage,Exporter,Importer,Trade Value
1980,"Iron and steel , Nonferrous metals",Capital goods,Argentina,Australia,"52,272",NaN
1981,"Iron and steel , Nonferrous metals",Capital goods,Argentina,Australia,"116,393",NaN
1982,"Iron and steel , Nonferrous metals",Capital goods,Argentina,Australia,"178,355",NaN
1983,"Iron and steel , Nonferrous metals",Capital goods,Argentina,Australia,"59,986",NaN
1984,"Iron and steel , Nonferrous metals",Capital goods,Argentina,Australia,"79,184",NaN
...,...,...,...,...,...,...
1996,Toys and Miscellaneous goods,Parts and Compornents,Argentina,Australia,636,NaN
1999,Toys and Miscellaneous goods,Parts and Compornents,Argentina,Australia,780,NaN
2009,Toys and Miscellaneous goods,Parts and Compornents,Argentina,Australia,"1,497",NaN
2010,Toys and Miscellaneous goods,Parts and Compornents,Argentina,Australia,"5,881",NaN
